The Turán number of $K_4^-$
=================================

In [7]:
from sage.algebras.flag_algebras import *

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
TG.exclude([k4,k4m])

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoK4m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))

11 106 8157


In [8]:
f=TGp(6, edges=[ [1,2,3], [2,3,4], [3,4,5], [4,5,1], [5,1,2],
               [1,3,0], [2,4,0], [3,5,0], [4,1,0], [5,2,0] ])
degree = TGp(3, edges=[ [0,1,2] ], ftype=[0])
# max_F = TGp.optimize_problem(1*f, 6, maximize=True, positives=[degree-2/7])
max_F = TGp.optimize_problem(f, 7, maximize=True, positives=[degree-0.285714285])

#around 23Gb memory required
#block sizes are [5, 95, 47, 388, 262, 191, 173, 148, 135, 124, 101, 95, 94, 72, -8157, -26]

Ftypes constructed in 62.83s
Block sizes done in 146.06s
Block sizes are [5, 95, 47, 388, 262, 191, 173, 148, 135, 124, 101, 95, 94, 72, -8157, -26]
Calculating product matrices for 14 ftypes and 8157 structures
Ftype on 1 points with edges=[] is complete: : 1it [04:03, 243.28s/it]


KeyboardInterrupt: 